In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
import pandas as pd
import time
import re
import os 
import re
import csv


In [43]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
URL = 'https://www.ankhang.vn/laptop-theo-hang.html'
driver.get(URL)

In [44]:
# get brand__laptop_links

link_laptop_brands = []

container_brands = driver.find_element(By.CSS_SELECTOR, '.filter-holder.bg-white.h-100 .filter-left')
lable_brands = container_brands.find_elements(By.CSS_SELECTOR, 'label')
print(f"Số hãng laptop tìm thấy: {len(lable_brands)}")
print(type(lable_brands[0]))

for lable in lable_brands: 
    print(f"lable_text : {lable.text}")
    link_str = lable.get_attribute('onclick')
    if link_str:
        print(link_str)
        start = link_str.find("'") + 1
        end = link_str.rfind("'")
        link = link_str[start:end]
        link_laptop_brands.append(link)
print(f"totals link laptop brands: {len(link_laptop_brands)}")


Số hãng laptop tìm thấy: 10
<class 'selenium.webdriver.remote.webelement.WebElement'>
lable_text : Laptop Acer (147)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=135'
lable_text : Laptop Asus (378)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=134'
lable_text : Laptop Dell (152)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=137'
lable_text : Laptop HP (515)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=128'
lable_text : Laptop Lenovo (600)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=165'
lable_text : Laptop MSI (122)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=199'
lable_text : Apple (2)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=329'
lable_text : Gigabyte (40)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=59'
lable_text : Laptop LG (9)
location.href='https://www.ankhang.vn/laptop-theo-hang.html?brand=2'
lable_text : Sam

In [4]:
# for idx, link in enumerate(link_laptop_brands):
#     print(f"in link: {link}")
#     try:
#         driver.get(link)

#         #wait for page loading
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.TAG_NAME, 'body'))
#         )
#         print(f"loaded link: {idx} scuccessfully")
        



#     except Exception as e:
#         print(f"error when crawl link: {idx}, error:{e}")
        


In [45]:
def crawl_href_item():
    link_items = []

    item_containers = driver.find_elements(By.CSS_SELECTOR, '.product_list_box .p-item-2021 .p-container')
    for item in item_containers:
        try:
            link_item = item.find_element(By.TAG_NAME, 'a').get_attribute('href')
            link_items.append(link_item)
            
        except Exception as e:
            print(f"loi khi lay href item: {e}")
    return link_items
    

def go_to_next_page():
    try:
        # lay block phan trang
        paging = driver.find_element(By.CSS_SELECTOR, '.paging')

        #lay trang hien tai
        curent_page_ele = paging.find_element(By.CSS_SELECTOR, '.current')
        current_page = int(curent_page_ele.text.strip())

        # lay danh sach trang
        page_nums = []
        page_num_elements =  paging.find_elements(By.TAG_NAME, 'a')
        for ele in page_num_elements:
            try:
                if ele.text.strip().isdigit():
                    page_nums.append(
                        {
                            'page_num' : int(ele.text.strip()),
                            'href' : ele.get_attribute('href')
                        }
                    )
            except Exception as e:
                print(f"loi khi lay href paging : {e}")
        
        # tim trang tiep theo
        next_page_num = None
        next_page_href = None

        for page_ele in page_nums:
            if page_ele['page_num'] > current_page:
                next_page_num = page_ele['page_num']
                next_page_href = page_ele['href']
                break
        
        return next_page_num, next_page_href
    except Exception as e:
        print(f"loi khi lay trang tiep theo: {e}")
        return None, None



In [46]:
def cleaning_text(text : str)-> str:
    """
     loai bo cac ki tu xuong dong khi chua ket thuc cau, xoa khoang trong thua.
    """
    # thay xuong dong thanh khoang trang khi chua ket thuc cau
    text = re.sub(r'(?<![.!?;:])\n(?!\n)', ' ', text)
    # xoa khoang trong thua
    text = re.sub(r'\n{2,}', '\n\n', text)
    
    return text.strip()


In [49]:
def crawl_detail_item():
    # lay link model item
    link_models = []
    current_url = driver.current_url
    current_model_name = None
    count = 1
    output_file = r"laptop.csv"

    while True: 
        try:
            model_containers = driver.find_elements(By.CSS_SELECTOR, '.choose_model.ver-1200px .owl-item.active')
            print(f"so model acitve tim thay: {len(model_containers)}")
            tmp_model = model_containers[-1]
            try:
                tmp_model_name = tmp_model.find_element(By.CSS_SELECTOR, '.container_check').text.strip()
                print(f"ten model moi: {tmp_model_name}")
                if current_model_name == tmp_model_name:
                    print("da load het model")
                    break
                current_model_name = tmp_model_name
            except Exception as e:
                print(f"khong lay duoc ten model: {e}")
                tmp_model_name = None
            
            if model_containers:
                if count == 1:
                    model_loop = model_containers
                else:
                    model_loop = [tmp_model]

                for model in model_loop:
                    price = "hihihihihihii"
                    # crawl price
                    price_ele = model.find_element(By.CSS_SELECTOR, '.price_model')
                    price = price_ele.text.strip()
                    print(f"gia model crawl duoc: {price}, kieu du lieu: {type(price)}")

                    link_str = model.find_element(By.CSS_SELECTOR, '.item_model.item').get_attribute('onclick')
                    print(f"_________________link_str__________________: {link_str}")
                    match = re.search(r"location\.href='([^']+)'", link_str or "")
                    if match:
                        print(f"parse duoc link_str: {link_str}")
                        relative_url = match.group(1)
                        detail_url = urljoin(current_url, relative_url)
                        
                        print(f"detail_url crawl duoc: {detail_url}")
                        # chua duoc crawl thi them moi
                        link_models.append({
                                            'detail_url':detail_url,
                                            'price': price
                                            })
                    else:
                        print("Không parse được URL:", link_str)
            try:
                # tim nut next model
                next_btn = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.CSS_SELECTOR, '.choose_model.ver-1200px .owl-next'))
                            )
                next_btn.click()
            
            except Exception as e:
                print(f"het model de load: {e}")
                break

                                            
            else:
                pass

            count += 1
            
        except Exception as e:
            print(f"san pham khong co model :{e}")

    # crawl chi tiet tung model
    if link_models:
        for model in link_models:
            try:
                driver.get(model['detail_url'])
                try:
                    # tim nut xem them thong tin
                    more_content_btn = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '.more_content a'))
                    )
                    driver.execute_script("arguments[0].click();", more_content_btn)

                    # # xu ly price 
                    price = model['price']
                    # if not price:

                    # cho noi dung hien thi 
                    detail_ele = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.CSS_SELECTOR, '#pro-desc'))
                    )
                    text = detail_ele.text
                    cleaned_text = cleaning_text(text)

                    combined_text = "Giá bán khuyến mãi đang được áp dụng là :" + price + " " + cleaned_text
                    print(f"____________crawl duoc thong tin: {combined_text}")
                    
                    # luu ket qua vao file csv 
                    with open(output_file, mode='a', encoding='utf-8') as f:
                        writer = csv.writer(f)
                        writer.writerow([model['detail_url'],price, cleaned_text])
                        
            
                except Exception as e:
                    print(f"khong co nut xem thong tin chi tiet ")
                    continue
            except Exception as e:
                print(f"loi khi crawl chi tiet model:{e}")
    
    return

            
        
        # # lay list phien ban tung model
        # for model in link_models:
        #     driver.get(model)
        #     WebDriverWait(driver, 10).until(
        #         EC.presence_of_element_located((By.TAG_NAME, 'body'))
        #     )
        #     link_versions = []
        #     for link in link_models:
        #         try:
        #             driver.get(link)
        #             version_ele = driver.find_element(By.CSS_SELECTOR, '.choose_version.clearfix .item-group')
        #             link_version_ele = version_ele.find_elements(By.TAG_NAME, 'a')
        #             for version in link_version_ele:
        #                 link_versions.append(version.get_attribute('href'))
        #         except Exception as e:
        #             print("item khong co version")
        #             continue
        #     print(f"model co {len(link_versions)} version: {link_versions}")
            
        #     # crawl chi tiet tung version
        #     for version in link_versions:
        #         driver.get(version)
        #         WebDriverWait(driver, 10).until(`
        #             EC.presence_of_element_located((By.TAG_NAME, 'body'))
        #         )
        #         print(f"dang crawl chi tiet version: {version} from link : {version}")



            





In [50]:
driver.get(link_laptop_brands[0])
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'body'))
)
link_items = crawl_href_item()
if link_items:
    print(f"tim thay {len(link_items)} link item")
    
    # crawl chi tiet item
    for link in link_items[:2]:
        driver.get(link)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        crawl_detail_item()


tim thay 20 link item
so model acitve tim thay: 2
ten model moi: Laptop Acer Gaming Aspire 7 A715-59G-55MD NH.QX6SV.005
gia model crawl duoc: 17.490.000 VNĐ, kieu du lieu: <class 'str'>
_________________link_str__________________: location.href='/laptop-gaming-acer-nitro-v-15-propanel-anv15-51-500a-nh.qnasv.008.html'
parse duoc link_str: location.href='/laptop-gaming-acer-nitro-v-15-propanel-anv15-51-500a-nh.qnasv.008.html'
detail_url crawl duoc: https://www.ankhang.vn/laptop-gaming-acer-nitro-v-15-propanel-anv15-51-500a-nh.qnasv.008.html
gia model crawl duoc: 19.790.000 VNĐ, kieu du lieu: <class 'str'>
_________________link_str__________________: location.href='/laptop-acer-gaming-aspire-7-a715-59g-55md-nh.qx6sv.005.html'
parse duoc link_str: location.href='/laptop-acer-gaming-aspire-7-a715-59g-55md-nh.qx6sv.005.html'
detail_url crawl duoc: https://www.ankhang.vn/laptop-acer-gaming-aspire-7-a715-59g-55md-nh.qx6sv.005.html
so model acitve tim thay: 2
ten model moi: Laptop Acer Gaming N

In [ ]:

driver.get(link_laptop_brands[0])
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'body'))
)
print(f"loaded link scuccessfully")
while True:
    try:
        next_page_num, next_page_href = go_to_next_page()
        
        # crawl href item
        link_items = crawl_href_item()
        if link_items:
            print(f"tim thay {len(link_items)} link item")
            crawl_detail_item()
            
            # crawl chi tiet item
            for link in link_items[:2]:
                driver.get(link)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )
                crawl_detail_item()
                
        # chuyen trang tiep theo
        if next_page_href:
            print(f"dang chuyen den trang {next_page_num} - {next_page_href}")
            driver.get(next_page_href)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'body'))
            )
        else:
            print(f"da het trang de crawl")
            break
        
    except Exception as e:
        print(f"loi khi crawl trang: {e}")
        

        


loaded link scuccessfully
tim thay 20 link item
san pham khong co model
san pham khong co model
san pham khong co model
san pham khong co model
san pham khong co model
san pham khong co model
khong co nut xem thong tin : Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x11d4103
	0x11d4144
	0xfde71d
	0x102a03d
	0x102a41b
	0x10717f2
	0x104c954
	0x106ee17
	0x104c706
	0x101da30
	0x101ed54
	0x14457b4
	0x144098a
	0x11fc392
	0x11ec4c8
	0x11f324d
	0x11dc478
	0x11dc63c
	0x11c67ca
	0x7696fcc9
	0x775b82ae
	0x775b827e

khong co nut xem thong tin : Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x11d4103
	0x11d4144
	0xfde71d
	0x102a03d
	0x102a41b
	0x10717f2
	0x104c954
	0x106ee17
	0x104c706
	0x101da30
	0x101ed54
	0x14457b4
	0x144098a
	0x11fc392
	0x11ec4c8
	0x11f324d
	0x11dc478
	0x11dc63c
	0x11c67ca
	0x7696fcc9
	0x775b82ae
	0x775b827e

Không parse được URL: location.href=''
link_str: location.href=''
san pham khong co model
san pham khong co mo

KeyboardInterrupt: 

In [51]:
df = pd.read_csv("laptop.csv")
df.head()

https://www.ankhang.vn/laptop-gaming-acer-nitro-v-15-propanel-anv15-51-500a-nh.qnasv.008.html  \
0  https://www.ankhang.vn/laptop-acer-gaming-nitr...                                              
1  https://www.ankhang.vn/laptop-gaming-msi-cybor...                                              
2  https://www.ankhang.vn/laptop-acer-aspire-go-1...                                              
3  https://www.ankhang.vn/laptop-acer-aspire-lite...                                              
4  https://www.ankhang.vn/laptop-acer-swift-lite-...                                              

   17.490.000 VNĐ  \
0  20.590.000 VNĐ   
1  23.590.000 VNĐ   
2  18.890.000 VNĐ   
3  18.790.000 VNĐ   
4  20.790.000 VNĐ   

  ĐẶC ĐIỂM NỔI BẬT Laptop Gaming Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 - Laptop hiệu năng cao, giá hấp dẫn Laptop Gaming Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 là một chiếc laptop gaming tầm trung được sản xuất bởi Acer, thuộc dòng Nitro V. Chiếc laptop này được trang bị CPU Intel Core i5-13420H, RAM 16GB, SSD 512GB, card đồ họa NVIDIA GeForce RTX 2050 đáp ứng tốt các tựa game phổ biến hiện nay. Màn hình 15.6 inch FHD IPS sắc nét với tốc độ làm mới lên tới 180Hz cho trải nghiệm cực mượt mà. Nhìn chung, Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 là sự lựa chọn phù hợp cho những ai đang tìm kiếm một chiếc laptop hiệu năng cao.\r\nLaptop Gaming Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 - Laptop hiệu năng cao, giá hấp dẫn Hiệu năng mạnh mẽ với CPU Intel® Core™ i5-13420H, RAM 16GB DDR5 và SSD 512GB Laptop Gaming Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 được trang bị bộ vi xử lý Intel® Core™ i5-13420H (8 nhân, 12 luồng), mang lại khả năng xử lý mượt mà các tác vụ đòi hỏi hiệu suất cao. Bạn hoàn toàn có thể sử dụng các phần mềm chuyên dụng như Photoshop, Premiere hay AutoCAD hay “chiến” các tựa game AAA đồ họa khủng, thiết kế đồ họa và lập trình. Với tốc độ xung nhịp lên đến 4.6 GHz, Acer Nitro V 15 ProPanel đảm bảo hiệu suất ổn định ngay cả khi chạy các ứng dụng nặng.\r\nAcer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 sở hữu bộ nhớ RAM 16GB DDR5 5200MHz tối ưu hóa khả năng đa nhiệm, xử lý đồng thời nhiều tác vụ mà không bị giật lag. Ổ cứng SSD 512GB PCIe NVMe cho tốc độ truyền tải dữ liệu cực nhanh, tăng tốc độ mở hệ điều hành và ứng dụng, giảm thiểu đáng kể thời gian chờ đợi. Máy hỗ trợ nâng cấp SSD lên tới 4TB mang lại không gian lưu trữ rộng rãi, đáp ứng nhu cầu của cả game thủ và người dùng chuyên nghiệp.\r\nLaptop Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 với cấu hình ổn định Trải nghiệm đồ họa mượt mà với NVIDIA® GeForce RTX™ 2050 Laptop Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 sẽ mang đến cho bạn trải nghiệm chơi game mượt mà nhờ card đồ họa NVIDIA® GeForce RTX™ 2050. Với công nghệ Ray Tracing tiên tiến, hình ảnh trong game trở nên sống động và chân thực hơn bao giờ hết, từng tia sáng, bóng đổ đều được tái hiện một cách tinh tế. Bạn có thể đắm mình vào thế giới ảo của các tựa game AAA như Cyberpunk 2077 hay Fortnite với chất lượng hình ảnh tuyệt đẹp và tốc độ khung hình mượt mà. Không chỉ dừng lại ở đó, GeForce RTX™ 2050 còn là một công cụ đắc lực cho các nhà sáng tạo nội dung. Việc chỉnh sửa video, thiết kế đồ họa 3D cũng trở nên dễ dàng hơn khả năng xử lý mạnh mẽ của card đồ họa này.\r\nAcer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 trang bị card đồ họa rời RTX 2050 Hệ thống tản nhiệt mạnh mẽ với 2 quạt và khả năng tùy chỉnh qua NitroSense™ Acer Nitro V 15 ProPanel ANV15-51-500A NH.QNASV.008 được trang bị hệ thống tản nhiệt hiệu quả, giúp máy hoạt động ổn định ngay cả khi bạn đang chơi game nặng hoặc làm việc với các phần mềm đòi hỏi hiệu suất cao. Với hai quạt làm mát thông minh cùng công nghệ quản lý nhiệt tiên tiến, máy tính sẽ luôn giữ được nhiệt độ lý tưởng, bảo vệ các linh kiện bên trong khỏi tình trạng quá nhiệt, nhờ đó kéo dài tuổi thọ cho máy.Thêm vào đó, NitroSense™ là ứng dụng hữu ích cho phép người dùng tùy chỉnh tốc độ 